In [12]:
# 
import plotly.graph_objects as go # Note: Requires Version 4.14.3 to run without dedacted errors
import numpy as np # Note: Requires Version: 1.21.5
from ipywidgets import widgets, FileUpload, VBox, IntText, Layout # Note: Requires Version: 7.5.1 to run properly
import os
from IPython.display import display, clear_output # Note: Requires Version 7.34.0 to run properly 
import pickle
import json

# Define all initial inputs
total_circles_input = widgets.IntText(description='Num circles: ', value=10)
seat_density_input = widgets.FloatText(description='Seat density: ', value=1)
offset_x_input = widgets.IntText(description='Radii center offset x: ', value=10)
offset_y_input = widgets.IntText(description='Radii center offset y: ', value=10)
radius_separation_distance = widgets.IntText(description='Radii separation: ', value=1)
marker_size_input = widgets.IntText(description='Marker Size: ', value=7)
image_file_source = widgets.Text(description='Img File Source: ', value='Image_Data/history_2.png', layout=Layout(width='auto'))
background_image_x_dim_size = widgets.IntText(description='Img_x_dim: ', value=50)
background_image_y_dim_size = widgets.IntText(description='Img_y_dim: ', value=50)

# Modify the description text field & input field sizes so all text is displayed and is of properly formatted size
radius_separation_distance.style.description_width = 'auto'
offset_y_input.style.description_width = 'auto'
offset_x_input.style.description_width = 'auto'
image_file_source.style.description_width = 'auto'

submit_dim_button = widgets.Button(description="Submit all inputs")

# Horizontal box packaging containers for each widgets.InText value input
horizontal_initial_inputs_row_1 = widgets.HBox([total_circles_input, seat_density_input])
horizontal_initial_inputs_row_2 = widgets.HBox([offset_x_input, offset_y_input, radius_separation_distance])
horizontal_initial_inputs_row_3 = widgets.HBox([marker_size_input])
horizontal_initial_inputs_row_4 = widgets.HBox([image_file_source, background_image_x_dim_size, background_image_y_dim_size])
horizontal_initial_inputs_row_5 = widgets.HBox([submit_dim_button])

# Display all horizontal boxes in specified order
display(horizontal_initial_inputs_row_1)
display(horizontal_initial_inputs_row_2)
display(horizontal_initial_inputs_row_3)
display(horizontal_initial_inputs_row_4)
display(horizontal_initial_inputs_row_5)


# dictionary for saved clicked points
gallery = {}


def on_submit_dim_button(_):
    
    total_circles = total_circles_input.value
    seat_density = seat_density_input.value
    offset_x = offset_x_input.value
    offset_y = offset_y_input.value
    radius_separation = radius_separation_distance.value
    marker_size = marker_size_input.value
    
    x_values = []
    y_values = []
    colors = []
    line_colors = []
    hovertext = []
    
    for i in range(total_circles):
        radius = i + 1
        circumference = 2 * np.pi * radius
        num_markers = int(np.ceil(circumference * seat_density))
        theta_values = np.linspace(0, 2 * np.pi, num_markers, endpoint=False)
        circle_x = offset_x + radius * np.cos(theta_values)
        circle_y = offset_y + radius * np.sin(theta_values)
        
        x_values.extend(circle_x)
        y_values.extend(circle_y)
        colors.extend(['rgba(0, 0, 0, 0)'] * num_markers)
        line_colors.extend(['black'] * num_markers)
        hovertext.extend([f"Seat {i+1}<br>X: {circle_x[i]}<br>Y: {circle_y[i]}" for i in range(num_markers)])
    
    scatter = go.Scatter(
        x=x_values,
        y=y_values,
        mode='markers',
        marker=dict(size=marker_size, color=colors, line=dict(color=line_colors, width=0.15)),
        hoverinfo='text',
        text=hovertext
    )
    
    f = go.FigureWidget([scatter])
    f.layout.hovermode = 'closest'

    # create our callback function
    def update_point(trace, points, selector):
        c = list(scatter.marker.color)
        s = list(scatter.marker.size)

        section_input = widgets.Text(description='Section:', value="", continuous_update=False)
        row_input = widgets.Text(description='Row:', value="", continuous_update=False)
        seat_input = widgets.Text(description='Seat:', value="", continuous_update=False)
        submit_button = widgets.Button(description="Submit")
        inputs_container = widgets.HBox([section_input, row_input, seat_input, submit_button])

        for i in points.point_inds:
            x_val = x_values[i]
            y_val = y_values[i]

            key_name = None
            for seat, seat_data in gallery.items():
                if seat_data['x'] == x_val and seat_data['y'] == y_val:
                    key_name = seat
                    break

            if key_name is not None:
                del gallery[key_name]

            else:
                c[i] = '#010C80'
                s[i] = 8
                display(inputs_container)

                def on_submit(_):
                    section = section_input.value.strip()
                    row = row_input.value.strip()
                    seat = seat_input.value.strip()
                    key_name = ""
                    if section:
                        key_name += f"Sec {section} "
                    if row:
                        key_name += f"Row {row} "
                    if seat:
                        key_name += f"Seat {seat}"
                    if key_name:
                        gallery[key_name] = {"name": key_name, "sec": section, "row": row, "seat number": seat,
                                             "x": x_val, "y": y_val, "color": '#010C80', "minted": False,
                                             "tokenID": None, "price": 0, "bought": False}
                        if f'Point {i}' in gallery:
                            del gallery[f'Point {i}']

                        with f.batch_update():
                            scatter.marker.color = c
                            scatter.marker.size = s

                    inputs_container.close()

                submit_button.on_click(on_submit)

                def on_keydown(change):
                    if change.new == '':
                        return
                    if change['name'] == 'value' and change['type'] == 'change':
                        if change['new'] == '\n':
                            on_submit(None)
                            return
                        if change['owner'] == section_input and row_input.value and seat_input.value:
                            on_submit(None)
                            return
                        if change['owner'] == row_input and section_input.value and seat_input.value:
                            on_submit(None)
                            return
                        if change['owner'] == seat_input and section_input.value and row_input.value:
                            on_submit(None)
                            return

                section_input.observe(on_keydown)
                row_input.observe(on_keydown)
                seat_input.observe(on_keydown)

        scatter.on_click(update_point)
                
                
        # define a function to handle the keydown event for input widgets
        def on_keydown(change):
            if change.new == '':
                return
            if change['name'] == 'value' and change['type'] == 'change':
                if change['new'] == '\n':
                    on_submit(None)
                    return
                if change['owner'] == section_input and row_input.value and seat_input.value:
                    on_submit(None)
                    return
                if change['owner'] == row_input and section_input.value and seat_input.value:
                    on_submit(None)
                    return
                if change['owner'] == seat_input and section_input.value and row_input.value:
                    on_submit(None)
                    return

        # add event listener for the "keydown" event to each input widget
        section_input.observe(on_keydown)
        row_input.observe(on_keydown)
        seat_input.observe(on_keydown)

    scatter.on_click(update_point)

    ##########

    def update_hovertext():
        new_hovertext = []
        new_colors = []
        for i in range(len(x_values)):
            x_val = x_values[i]
            y_val = y_values[i]
            found_in_gallery = False
            for key_name, point_info in gallery.items():
                if point_info['x'] == x_val and point_info['y'] == y_val:
                    new_hovertext.append(f"name: {point_info['name']}<br>sec: {point_info['sec']}<br>row: {point_info['row']}<br>seat number: {point_info['seat number']}<br>X: {x_val}<br>Y: {y_val}")
                    new_colors.append(point_info['color'])
                    found_in_gallery = True
                    break
            if not found_in_gallery:
                new_hovertext.append(f"Point {i}<br>X: {x_val}<br>Y: {y_val}")
                new_colors.append('#B2B4B8')

        scatter.hovertext = tuple(new_hovertext)
        scatter.marker.color = tuple(new_colors)

    update_hovertext_button = widgets.Button(description="Update Hovertext")

    def on_button_click(update_hovertext_button):
        update_hovertext()

    update_hovertext_button.on_click(on_button_click)

    ##########


#     xmax = np.max(x) + 25
#     xmin = np.min(x) - 25
#     ymax = np.max(y) + 25
#     ymin = np.min(y) - 25



#     f.update_layout(xaxis_range=[xmin, xmax], yaxis_range=[ymin, ymax], xaxis_autorange=False, yaxis_autorange=False)
#     f.update_layout(
#         title='Venue Builder - Section Constructor',
#         xaxis_title='Aisle',
#         yaxis_title='Row'
#     )
    
    #newly added background image code as of 05/26/2023
    f.update_layout(
        images=[dict(
            source=image_file_source.value,
            xref='x',
            yref='y',
            x=0,
            y=background_image_y_dim_size.value,
            sizex=background_image_x_dim_size.value, # size of the image in x-dir
            sizey=background_image_y_dim_size.value, # size of image in the y-dir
            sizing='stretch', 
            opacity=1, 
            layer='below'
        )]
    )
    
    ## test
#     f.update_layout(
#     width=800,
#     height=800
#     )
    ## test

#     f.update_layout(
#     width=800, height=800,  # Adjust the figure size as desired
#     xaxis=dict(range=[0, 50]),
#     yaxis=dict(range=[0, 50])
#     )


    f.update_layout(
    width=800, height=800,  # Adjust the figure size as desired
    xaxis=dict(range=[0, background_image_x_dim_size.value]),
    yaxis=dict(range=[0, background_image_y_dim_size.value])
    )
    
    
    f



    def view_gallery(): 
        return gallery

    view_gallery_output = widgets.Output()

    def on_view_gallery_button_click(view_gallery_button):
        with view_gallery_output:
            display(view_gallery())
            clear_output(wait=True)

    view_gallery_button = widgets.Button(description="View Gallery")
    view_gallery_button.on_click(on_view_gallery_button_click)


    display(f)
    #display(update_gallery_button)
    # display(widgets.HBox([update_hovertext_button, clear_button]))
    #display(view_gallery_button)
    #display(view_gallery_output)
    
    # create uploader instance for the save/load callback functions with FileUpload()
    uploader = FileUpload(accept=".txt")
    
    
    # save gallery as a text file with button callback function    
    def save_as_text(_):
        global gallery 
        display(uploader)

        # def save_as_text_button_click(sender):
        #     filename = next(iter(uploader.value))
        #     basename, extension = os.path.splitext(filename)
        #     basename = basename.replace('_text', '')
        #     txt_filename = os.path.join('txt', f'{basename}_text.txt')
        #     try:
        #         with open(txt_filename, 'w', encoding='utf-8') as txt_file:
        #             for key, value in gallery.items():
        #                 txt_file.write(f"{key}: {value}\n")
        #         print(f"'gallery' dictionary saved as text to {txt_filename}")
        #     except Exception as e:
        #         print(f"Error saving gallery as text: {e}")
        
        def save_as_text_button_click(sender):
            filename = next(iter(uploader.value))
            basename, extension = os.path.splitext(filename)
            basename = basename.replace('_text', '')
            txt_filename = os.path.join('txt', f'{basename}_text.txt')
            try:
                with open(txt_filename, 'w', encoding='utf-8') as txt_file:
                    txt_file.write(f"gallery = {{\n")
                    for key, value in gallery.items():
                        txt_file.write(f"    '{key}': {value},\n")
                    txt_file.write(f"}}\n")
                print(f"'gallery' dictionary saved as text to {txt_filename}")
            except Exception as e:
                print(f"Error saving gallery as text: {e}")

        # save button generation
        save_as_text_button = widgets.Button(description="Save as Text")
        try: 
            save_as_text_button.on_click(save_as_text_button_click)
        except Exception as e: 
            print(f"No file selected from upload widget: {e}") 

        display(save_as_text_button)
        
    
    # save gallery button callback function
    def save_gallery(_):
        global gallery
        display(uploader)
        def save_gallery_button_click(sender):
            #nonlocal gallery
            filename = next(iter(uploader.value))
            try:
                # save as a pickled binary file
                with open(f'txt/bin/{filename}', 'wb') as bin_file: 
                    pickle.dump(gallery, bin_file)
                print(f"'gallery' dictionary saved to txt/bin/{filename}")
                
            # print exception if file does not save as bin and/or txt file    
            except Exception as e: 
                print(f"Error saving gallery: {e}")
                
        # save button generation
        save_gallery_button = widgets.Button(description="Save Gallery")
        try: 
            save_gallery_button.on_click(save_gallery_button_click)
        except Exception as e: 
            print(f"No file selected from Upload () widget: {e}")
        display(save_gallery_button)
    
    # load gallery button callback function 
    def load_gallery(_): 
        global gallery
        display(uploader)
        def load_gallery_button_click(sender): 
            global gallery
            #nonlocal gallery
            filename = next(iter(uploader.value))
            try:
                with open(f'txt/bin/{filename}', 'rb') as file: 
                    gallery = pickle.load(file, encoding='latin-1')
                print(f"'gallery' dictionary loaded from {filename}")
            except Exception as e:
                print(f"Error loading gallery: {e}")
                
        #load button generation
        load_gallery_button = widgets.Button(description="Load Gallery")
        try: 
            load_gallery_button.on_click(load_gallery_button_click)
        except Exception as e: 
                print(f"No file selected from Upload () widget: {e}")
        display(load_gallery_button)
                            
                                  
    # create on_click action (widget on_click executes/runs above functions)
    save_gallery_button = widgets.Button(description="Save File")
    load_gallery_button = widgets.Button(description="Open File")
    save_as_text_button = widgets.Button(description="Save as Text")
    
    save_gallery_button.on_click(save_gallery)
    load_gallery_button.on_click(load_gallery)
    save_as_text_button.on_click(save_as_text)
    
    # create horizontal box to hold the buttons & layout format
    buttons_horizontal_box = widgets.HBox([save_gallery_button, load_gallery_button, save_as_text_button])
    # buttons_horizontal_box.layout.justify_content = 'space-between'
    # buttons_horizontal_box.layout.align_items = 'center'
    
    # display horizontal button box
    display(buttons_horizontal_box) 
    
    #display(save_gallery_button, load_gallery_button, save_as_json_button)
    
    
    # button that clears all output
    
    clear_button = widgets.Button(description="Clear Output")
    
    def clear_button_on_click(clear_button): 
        clear_output(wait=True)
    
    clear_button.on_click(clear_button_on_click)
    
    display(widgets.HBox([update_hovertext_button, clear_button]))
    
submit_dim_button.on_click(on_submit_dim_button)

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': [rgba(0, 0, 0, 0), rgba(0, 0, 0, 0), rgba(0, 0,
                                   0, 0), ..., rgba(0, 0, 0, 0), rgba(0, 0, 0, 0),
                                   rgba(0, 0, 0, 0)],
                         'line': {'color': [black, black, black, ..., black,
                                            black, black],
                                  'width': 0.15},
                         'size': 7},
              'mode': 'markers',
              'text': [Seat 1<br>X: 11.0<br>Y: 10.0, Seat 2<br>X:
                       10.707106781186548<br>Y: 10.707106781186548, Seat 3<br>X:
                       10.0<br>Y: 11.0, ..., Seat 74<br>X: 19.694002659393306<br>Y:
                       7.545145128592008, Seat 75<br>X: 19.863613034027225<br>Y:
                       8.354054097192654, Seat 76<br>X: 19.965844930066698<br>Y:
                       9.174206545276672],
              'type': 'scatter',
      

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': [rgba(0, 0, 0, 0), rgba(0, 0, 0, 0), rgba(0, 0,
                                   0, 0), ..., rgba(0, 0, 0, 0), rgba(0, 0, 0, 0),
                                   rgba(0, 0, 0, 0)],
                         'line': {'color': [black, black, black, ..., black,
                                            black, black],
                                  'width': 0.15},
                         'size': 7},
              'mode': 'markers',
              'text': [Seat 1<br>X: 11.0<br>Y: 10.0, Seat 2<br>X: 10.5<br>Y:
                       10.86602540378444, Seat 3<br>X: 9.5<br>Y: 10.86602540378444,
                       ..., Seat 49<br>X: 19.32472229404356<br>Y: 6.38758333812847,
                       Seat 50<br>X: 19.697969360350093<br>Y: 7.560862798916224,
                       Seat 51<br>X: 19.924205096719355<br>Y: 8.77111709335286],
              'type': 'scatter',
              'uid': 'a57ad2b6-5c28

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': [rgba(0, 0, 0, 0), rgba(0, 0, 0, 0), rgba(0, 0,
                                   0, 0), ..., rgba(0, 0, 0, 0), rgba(0, 0, 0, 0),
                                   rgba(0, 0, 0, 0)],
                         'line': {'color': [black, black, black, ..., black,
                                            black, black],
                                  'width': 0.15},
                         'size': 7},
              'mode': 'markers',
              'text': [Seat 1<br>X: 11.0<br>Y: 10.0, Seat 2<br>X:
                       10.766044443118979<br>Y: 10.64278760968654, Seat 3<br>X:
                       10.17364817766693<br>Y: 10.984807753012207, ..., Seat
                       80<br>X: 19.73695423877779<br>Y: 7.721464910968626, Seat
                       81<br>X: 19.882804237803484<br>Y: 8.473507157811259, Seat
                       82<br>X: 19.970658011837408<br>Y: 9.234507471635048],
              